# Lesson 34 - Decision Trees

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator

spark = SparkSession.builder.getOrCreate()

## Introduction to Decision Trees

A **decision tree classifier** is a rule-based classifier that applies a divide-and-conquer strategy to classification. A decision tree model generates a prediction for an observation by applying a sequence of logical if-else tests. Each test checks to see if one particular feature is above or below a particular threshold value. The results of each test determine the next test to be applied, and ultimately, the label value to be predicted for the observations. 

Every decision tree has a **depth**. This can be thought of as the maximum number of questions that the tree will ask before determining a classification.  Note that depending on the answers to the questions, a decision tree will sometimes be able to determine a classification after asking fewer questions than its depth. For example, suppose that a decision tree occasionally needs to ask 6 questions to determine a classification, but it is always able to determine a classification after 6 or fewer questions. Then its depth is 6.

## Construction of a Decision Tree
A decision tree is constructed from training data as follows:

1. All observations are collected into a single **root node**. 
2. The root node is split into two pieces. This split is performed by selecting a single feature and a threshold upon which to split that feature. Observations with a feature value less than the threshold are placed into the **left node** and observations with a feature value greater than the threshold are placed into the **right node**. The training algorithm selects the feature and threshold that achieve the best separation in the label values. 
3. The nodes created in step each split into two nodes. The training algorithm searches for the optimal feature to split each of these nodes on separately. 
4. This splitting process continues until some stopping condition is satisfied. The unsplit nodes at the bottom of the tree are referred to as leaf nodes. 

Three common stopping conditions are described below. 

- Continue splitting nodes until the observations in any given leaf node all have the same label. 
- Continue splitting until a pre-specified maximum depth is obtained. 
- A minimum number of observations per node is sometimes specified. In this case, a node will not be split if doing so would cause one or both of the new nodes to have fewer observations than this minimum. 

![iris-tree](https://drbeane.github.io/files/images/417/iris_Tree.png)

## Load and Prepare Data

To demonstrate how to use PySpark to create an analyze a decision tree model, we will return to the [Titanic dataset](https://www.kaggle.com/c/titanic/data).

In [0]:
titanic = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .schema(
        'survived BYTE, pclass BYTE, name STRING, sex STRING, '
        'age FLOAT, ss_abd BYTE, pc_abd BYTE, fare FLOAT'
    )
    .csv('/FileStore/tables/titanic.txt')
)
 
titanic.printSchema()

root
-- survived: byte (nullable = true)
-- pclass: byte (nullable = true)
-- name: string (nullable = true)
-- sex: string (nullable = true)
-- age: float (nullable = true)
-- ss_abd: byte (nullable = true)
-- pc_abd: byte (nullable = true)
-- fare: float (nullable = true)

In [0]:
titanic.show(10, truncate=False)

+--------+------+--------------------------------------------------+------+----+------+------+-------+
survived|pclass|name |sex |age |ss_abd|pc_abd|fare |
+--------+------+--------------------------------------------------+------+----+------+------+-------+
0 |3 |Mr. Owen Harris Braund |male |22.0|1 |0 |7.25 |
1 |1 |Mrs. John Bradley (Florence Briggs Thayer) Cumings|female|38.0|1 |0 |71.2833|
1 |3 |Miss. Laina Heikkinen |female|26.0|0 |0 |7.925 |
1 |1 |Mrs. Jacques Heath (Lily May Peel) Futrelle |female|35.0|1 |0 |53.1 |
0 |3 |Mr. William Henry Allen |male |35.0|0 |0 |8.05 |
0 |3 |Mr. James Moran |male |27.0|0 |0 |8.4583 |
0 |1 |Mr. Timothy J McCarthy |male |54.0|0 |0 |51.8625|
0 |3 |Master. Gosta Leonard Palsson |male |2.0 |3 |1 |21.075 |
1 |3 |Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson |female|27.0|0 |2 |11.1333|
1 |2 |Mrs. Nicholas (Adele Achem) Nasser |female|14.0|1 |0 |30.0708|
+--------+------+--------------------------------------------------+------+----+------+------+-------+
only showing top 10 rows

In [0]:
N = titanic.count()
print(N)

887

### Distribution of Label Values

To serve as a baseline against which we can compare our model, we will check the distribution of the label values.

In [0]:
(
    titanic
    .select('survived')
    .groupby('survived')
    .agg(
        expr('COUNT(*) as count'), 
        expr(f'ROUND(COUNT(*)/{N},4) as prop')
    )
    .show()
)

+--------+-----+------+
survived|count| prop|
+--------+-----+------+
 1| 342|0.3856|
 0| 545|0.6144|
+--------+-----+------+

### Identify Numerical and Categorical Features

We need to create lists specifying the names of our numerical features and our categorical features.

In [0]:
num_features = ['age', 'ss_abd', 'pc_abd']
cat_features = ['pclass', 'sex']
features = num_features + cat_features

### Preprocessing

We will now create stages to be used in a pre-processing pipeline. It is important to note that the PySpark implementation of the decision tree algorithm **DOES NOT** require categorical variables to be processed using one-hot encoding. In fact, the decision tree models in PySpark tend to perform better if one-hot encoding is not applied to categorical variables. It is, however, still necessary to perform integer encoding on categorical variables.

In [0]:
ix_features = [c + '_ix' for c in cat_features]

feature_indexer = StringIndexer(inputCols=cat_features, outputCols=ix_features)

assembler = VectorAssembler(inputCols=num_features + ix_features, outputCol='features')

In [0]:
preprocessor = Pipeline(stages=[feature_indexer, assembler]).fit(titanic)
train = preprocessor.transform(titanic)
train.select('features', 'survived').show(10, truncate=False)

+----------------------+--------+
features |survived|
+----------------------+--------+
(5,[0,1],[22.0,1.0]) |0 |
[38.0,1.0,0.0,1.0,1.0]|1 |
(5,[0,4],[26.0,1.0]) |1 |
[35.0,1.0,0.0,1.0,1.0]|1 |
(5,[0],[35.0]) |0 |
(5,[0],[27.0]) |0 |
(5,[0,3],[54.0,1.0]) |0 |
[2.0,3.0,1.0,0.0,0.0] |0 |
[27.0,0.0,2.0,0.0,1.0]|1 |
[14.0,1.0,0.0,2.0,1.0]|1 |
+----------------------+--------+
only showing top 10 rows

### Evaluator

We will create an accuracy evaluator for use in scoring our models.

In [0]:
accuracy_eval = MulticlassClassificationEvaluator(
    predictionCol='prediction', labelCol='survived', metricName='accuracy')

## Decision Tree (maxDepth=3)

The first decision tree model we will consider will have a maximum depth of 3 and will be required to contain at least 8 observations in each of its leaf nodes.

In [0]:
dtree_03 = DecisionTreeClassifier(featuresCol='features', labelCol='survived', 
                               maxDepth=3, minInstancesPerNode=8, seed=1)

dtree_model_03 = dtree_03.fit(train)

### Training Score

In [0]:
train_pred_03 = dtree_model_03.transform(train)
acc_03 = accuracy_eval.evaluate(train_pred_03)
print('Training Accuracy:', acc_03)

Training Accuracy: 0.8094701240135288

### Cross Validation Score

In [0]:
cv = CrossValidator(estimator=dtree_03, estimatorParamMaps=[{}], 
                    evaluator=accuracy_eval, numFolds=10, parallelism=6)

cv_model = cv.fit(train)

cv_score_03 = cv_model.avgMetrics[0]

print('\nCross-Validation Estimate of Out-Of-Sample Performance:', cv_score_03)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

Cross-Validation Estimate of Out-Of-Sample Performance: 0.7900755115896763

### Visualizing the Tree Structure

Every `DecisionTreeModel` object has a `toDebugString` attribute that stores a string describing the structure of the decision tree. We can print this to see the rules that the decision tree uses to generate its predictions.

In [0]:
print(dtree_model_03.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_b8cca970ea6b, depth=3, numNodes=11, numClasses=2, numFeatures=5
 If (feature 4 in {0.0})
 If (feature 0 <= 13.5)
 If (feature 1 <= 2.5)
 Predict: 1.0
 Else (feature 1 > 2.5)
 Predict: 0.0
 Else (feature 0 > 13.5)
 Predict: 0.0
 Else (feature 4 not in {0.0})
 If (feature 3 in {0.0,2.0})
 If (feature 3 in {2.0})
 Predict: 1.0
 Else (feature 3 not in {2.0})
 Predict: 0.0
 Else (feature 3 not in {0.0,2.0})
 Predict: 1.0

In [0]:
pd.DataFrame([features], index=['feature'])

Out[14]:

,0,1,2,3,4
feature,age,ss_abd,pc_abd,pclass,sex


[sex == male]
                     +--------------+--------------+
                     |                             |
                [age < 13.5]            [pclass in {2nd or 3rd}]
            +--------+--------+           +--------+--------+
            |                 |           |                 |
      [ss_abd < 2.5]          0    [pclass is 2nd]          1  
      +-----+-----+                 +-----+-----+
      |           |                 |           |
      1           0                 1           0

### Feature Importance

In [0]:
pd.DataFrame({
    'feature':features,
    'importance':dtree_model_03.featureImportances
})

Out[15]:

,feature,importance
0,age,0.066189
1,ss_abd,0.089379
2,pc_abd,0.000000
3,pclass,0.170921
4,sex,0.673510


## Decision Tree (maxDepth = 20)

We will now consider a decision tree model with a maximum depth of 20 and with no minimum requirement for the number of observations in its leaf nodes.

In [0]:
dtree_20 = DecisionTreeClassifier(featuresCol='features', labelCol='survived', 
                               maxDepth=20, minInstancesPerNode=1, seed=1)

dtree_model_20 = dtree_20.fit(train)

### Training Score

In [0]:
train_pred_20 = dtree_model_20.transform(train)

acc_20 = accuracy_eval.evaluate(train_pred_20)

print('Training Accuracy:', acc_20)

Training Accuracy: 0.9007891770011274

### Cross-Validation Score

In [0]:
cv = CrossValidator(estimator=dtree_20, estimatorParamMaps=[{}], 
                    evaluator=accuracy_eval, numFolds=10, parallelism=6)

cv_model = cv.fit(train)

cv_score_20 = cv_model.avgMetrics[0]

print('\nCross-Validation Estimate of Out-Of-Sample Performance:', cv_score_20)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

Cross-Validation Estimate of Out-Of-Sample Performance: 0.7837536607269798

### Feature Importance

**Notice that the feature importance sums up to 1**

In [0]:
pd.DataFrame({
    'feature':features,
    'importance':dtree_model_20.featureImportances
})

Out[19]:

,feature,importance
0,age,0.251470
1,ss_abd,0.119491
2,pc_abd,0.047757
3,pclass,0.148970
4,sex,0.432312


## Comparing Models

We will close this lesson by comparing the results for the two decision tree models that we have constructed.

In [0]:
print('MaxDepth = 3')
print('Training Score:', acc_03)
print('CrossVal Score:', cv_score_03)

print()
print('MaxDepth = 20')
print('Training Score:', acc_20)
print('CrossVal Score:', cv_score_20)

MaxDepth = 3
Training Score: 0.8094701240135288
CrossVal Score: 0.7900755115896763

MaxDepth = 20
Training Score: 0.9007891770011274
CrossVal Score: 0.7837536607269798

### One hot encoding vs not one hot encoding

In spark, ability to group classes into a single question.